# Tutorial: CUDS API

This tutorial introduces the CUDS API.
The code given here is based on [this](https://github.com/simphony/osp-core/blob/master/examples/api_example.py) example.

## Background

CUDS stands for Common Universal Data Structure, and it is used to uniformly represent an ontological individual. In the python implementation of OSP-core, it means that every ontological individual is an instance of the `Cuds` class.

Conceptually, we consider CUDS objects as containers. The content of a CUDS object consists of other CUDS objects. This means that a CUDS is a [recursive data structure](https://en.wikipedia.org/wiki/Recursive_data_type). There are various types of relationships between a container and its contained objects. We will see this later on.

As a data structure, CUDS exposes an API that provides the functionalities *Create*, *Read*, *Update* and *Delete* (or simply [CRUD](https://en.wikipedia.org/wiki/Create,_read,_update_and_delete)).
Except *Create*, all the other functionalities are supported by the methods `add`, `get`, `remove`, `update` and `iter`, that are defined in the class `Cuds`. Here, we cover all of the methods except `update` since it is used to synchronize between two or more data sources, and in this tutorial we will only use one, that is the data we will create through our python code on the fly. Check out the [wrapper tutorial](multiple-wrappers.ipynb) to see the `update` method in action.

Every CUDS object is related to an ontological concept via the ontological `is a` relation.
In the python implementation of OSP-core, all ontological concepts are instances of the `OntologyEntity` class.
The ontological concept that can be used to *Create* CUDS objects are instances of `OntologyClass`, a subclass of `OntologyEntity`.
The ontological concepts are organized in namespaces.

## Let's get hands on
We start by importing the example namespace from OSP-core. It consists of concepts that were automatically generated from the [dummy city ontology](https://github.com/simphony/osp-core/blob/master/osp/core/ontology/docs/city.ontology.yml). There is [another tutorial](#) on the ontology YAML file. However, it is not important for the purposes of this tutorial.

Before we can start, you have to install the city ontology. Use out tool pico for this:

In [1]:
!pico install city

INFO 2020-08-06 08:03:23,850 [osp.core.ontology.installation]: Will install the following namespaces: ['city']
INFO 2020-08-06 08:03:23,880 [osp.core.ontology.yml.yml_parser]: Parsing YAML ontology file /mnt/c/Users/urba/Desktop/repos/osp-core/osp/core/ontology/docs/city.ontology.yml
INFO 2020-08-06 08:03:23,892 [osp.core.ontology.yml.yml_parser]: You can now use `from osp.core.namespaces import city`.
INFO 2020-08-06 08:03:23,892 [osp.core.ontology.parser]: Loaded 367 ontology triples in total
INFO 2020-08-06 08:03:23,927 [osp.core.ontology.installation]: Installation successful


Now you can start coding:

In [2]:
# If you just installed the ontology from within this notebook and this line doesn't work, please restart the kernel and run this cell again.
from osp.core.namespaces import city

Let's create a CUDS object that represents the city of Freiburg:

In [3]:
c = city.City(name="Freiburg", coordinates=[47, 7])

The variable `c` is assigned with a newly created CUDS object. This object was initialized with a name (Freiburg) and coordinates (\[47, 7\]). To understand why these two arguments were necessary, we'll have to take a look at the ontology. This will be explained in another tutorial.

Each CUDS object has a unique identifier (UID) which can be accessed:

In [4]:
print("uid of c: " + str(c.uid))

uid of c: 34ffc443-9287-46de-b2aa-477a57f68a2f


The type of an object can be queried as well:

In [5]:
print("type of c: " + str(c.oclass))

type of c: city.City


Let's add two citizens to our city object:

In [6]:
p1 = city.Citizen(name="Peter")
p2 = city.Citizen(name="Anne")
c.add(p1, rel=city.hasInhabitant)
c.add(p2, rel=city.hasInhabitant)

<city.Citizen: 088c08f9-7961-4586-b908-23170fb0f059,  CoreSession: @0x7f6c08683b20>

Note that the relationship type between the city and its two citizens in this case is 'HAS_INHABITANT'. In our context, this means that Anne and Peter are Freiburg inhabitants.

Next, we would like to iterate over the objects contained in the city object. We do so by using the `iter()` function:


In [7]:
for el in c.iter():
    print("uid: " + str(el.uid))

uid: 703ec4ba-e89b-4e79-8985-5b6d31223328
uid: 088c08f9-7961-4586-b908-23170fb0f059


We can `get()` an object from a container if we have a UID of one of its immediate contained objects:

In [8]:
print(c.get(p1.uid))

city.Citizen: 703ec4ba-e89b-4e79-8985-5b6d31223328


We can also filter the contained objects by type: 

In [9]:
print(c.get(oclass=city.Citizen))

[<city.Citizen: 703ec4ba-e89b-4e79-8985-5b6d31223328,  CoreSession: @0x7f6c08683b20>, <city.Citizen: 088c08f9-7961-4586-b908-23170fb0f059,  CoreSession: @0x7f6c08683b20>]


We remove objects using the `remove()` function:

In [10]:
c.remove(p1)
# c.remove(p1.uid) also works!

Let's close this tutorial by adding some neighbourhoods in a loop:

In [11]:
for i in range(6):
    c.add(city.Neighborhood(name="neighbourhood %s" % i))